In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, tree
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [81]:
_diseases = ["stroke", "heart_attack", "diabetes", "cancer", "kidney_disease", "ang_or_chd"]

In [82]:
#Create dataframes containing data for people who did and didn't develop x diseases
#Probably don't merge data frames (could be done at a later stage but probably doesn't work in our timeframe, requires too much data cleaning)

In [83]:
#Create a model to predict based off all of the features in the dataset for each disease
#Proabably use a neural network? Some kind of ensemble learning method

In [84]:
#Create models to impute missing input features based on the features that the user does provide
#Develop a confidence interval for the missing values, create an input vector with each quartile (?) from the confidence interval

In [85]:
#Take in user input
#Predict likelihood of development of each disease
#Use each input vector if we had to create a confidence interval imputation for missing data
#Create a confidence interval for likelhood of developing disease

In [86]:
#Determine which of the features contributed the most (?) to the likelihood of the more likely diseases
#Make recommendations for how to reduce the likelhiood of the diseases based on these features

In [87]:
df = pd.read_csv("2015.csv")
df = df[["BPHIGH4", "BPMEDS", "SEX", "TOLDHI2", "CVDINFR4", "CVDSTRK3", "DIABETE3", "WEIGHT2", "HEIGHT3", "SMOKE100", "SMOKDAY2", "DRNK3GE5", "_AGE80", "_FRUTSUM", "_VEGESUM", "_PA150R2", "_RACE", "CHCOCNCR", "CHCKIDNY", "CVDCRHD4"]]
df.head()

,BPHIGH4,BPMEDS,SEX,TOLDHI2,CVDINFR4,CVDSTRK3,DIABETE3,WEIGHT2,HEIGHT3,SMOKE100,SMOKDAY2,DRNK3GE5,_AGE80,_FRUTSUM,_VEGESUM,_PA150R2,_RACE,CHCOCNCR,CHCKIDNY,CVDCRHD4
0,1.0,1.0,2.0,1.0,2.0,2.0,3.0,280.0,510.0,1.0,3.0,NaN,63.0,50.0,217.0,3.0,1.0,2.0,2.0,2.0
1,3.0,NaN,2.0,2.0,2.0,2.0,3.0,165.0,508.0,1.0,1.0,NaN,52.0,24.0,78.0,1.0,1.0,2.0,2.0,2.0
2,3.0,NaN,2.0,1.0,7.0,1.0,3.0,158.0,511.0,NaN,NaN,NaN,71.0,NaN,NaN,9.0,1.0,1.0,2.0,2.0
3,1.0,1.0,2.0,1.0,2.0,2.0,3.0,180.0,507.0,2.0,NaN,NaN,63.0,100.0,20.0,3.0,1.0,1.0,2.0,2.0
4,3.0,NaN,2.0,2.0,2.0,2.0,3.0,142.0,504.0,2.0,NaN,NaN,61.0,NaN,200.0,3.0,1.0,2.0,2.0,2.0


In [11]:
df = df[df["HEIGHT3"] != 7777.0]
df = df[df["HEIGHT3"] != 9999.0]
df = df[df["WEIGHT2"] != 7777.0]
df = df[df["WEIGHT2"] != 9999.0]

In [12]:
df.describe()

,BPHIGH4,BPMEDS,SEX,TOLDHI2,CVDINFR4,CVDSTRK3,DIABETE3,WEIGHT2,HEIGHT3,SMOKE100,SMOKDAY2,DRNK3GE5,_AGE80,_FRUTSUM,_VEGESUM,_PA150R2,_RACEGR3
count,412999.000000,166838.000000,413000.000000,358034.000000,413000.000000,413000.000000,412993.000000,407685.000000,407369.000000,400846.000000,175276.000000,201379.000000,413000.000000,3.755700e+05,3.689370e+05,413000.000000,413000.000000
mean,2.206155,1.173641,1.560460,1.623659,1.965128,1.971663,2.756938,196.126544,555.154678,1.598976,2.432849,67.468837,55.433254,1.361374e+02,1.943727e+02,2.546293,1.652012
std,1.027882,0.445738,0.496332,0.718881,0.426887,0.334988,0.714990,397.807382,562.901366,0.701376,0.881476,36.053693,17.132764,1.375521e+02,1.549042e+02,2.409574,1.472622
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,50.000000,204.000000,1.000000,1.000000,1.000000,18.000000,5.397605e-79,5.397605e-79,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,3.000000,145.000000,504.000000,1.000000,2.000000,77.000000,43.000000,5.700000e+01,1.100000e+02,1.000000,1.000000
50%,3.000000,1.000000,2.000000,2.000000,2.000000,2.000000,3.000000,172.000000,507.000000,2.000000,3.000000,88.000000,58.000000,1.000000e+02,1.680000e+02,2.000000,1.000000
75%,3.000000,1.000000,2.000000,2.000000,2.000000,2.000000,3.000000,200.000000,510.000000,2.000000,3.000000,88.000000,69.000000,2.000000e+02,2.430000e+02,3.000000,1.000000
max,9.000000,9.000000,2.000000,9.000000,9.000000,9.000000,9.000000,9501.000000,9503.000000,9.000000,9.000000,99.000000,80.000000,1.500000e+04,1.992900e+04,9.000000,9.000000


In [13]:
def fix_height(height):
    if height <= 711:
        feet = height // 100
        inches = height % 100 + 12 * feet
    else:
        meters = height // 100
        centimeters = height % 100 + 12 * meters
        inches = centimeters / 2.54
    return inches
        

In [14]:
def fix_weight(weight):
    if weight <= 999:
        lbs = weight
    else:
        kilos = weight - 9000
        lbs = kilos / 2.2
    return lbs

In [15]:
df["height"] = df["HEIGHT3"].apply(fix_height)
df["weight"] = df["WEIGHT2"].apply(fix_weight)
df.drop(["HEIGHT3", "WEIGHT2"], axis = 1, inplace = True)
df.head()

,BPHIGH4,BPMEDS,SEX,TOLDHI2,CVDINFR4,CVDSTRK3,DIABETE3,SMOKE100,SMOKDAY2,DRNK3GE5,_AGE80,_FRUTSUM,_VEGESUM,_PA150R2,_RACEGR3,height,weight
0,1.0,1.0,2.0,1.0,2.0,2.0,3.0,1.0,3.0,NaN,63.0,50.0,217.0,3.0,1.0,70.0,280.0
1,3.0,NaN,2.0,2.0,2.0,2.0,3.0,1.0,1.0,NaN,52.0,24.0,78.0,1.0,1.0,68.0,165.0
2,3.0,NaN,2.0,1.0,7.0,1.0,3.0,NaN,NaN,NaN,71.0,NaN,NaN,9.0,1.0,71.0,158.0
3,1.0,1.0,2.0,1.0,2.0,2.0,3.0,2.0,NaN,NaN,63.0,100.0,20.0,3.0,1.0,67.0,180.0
4,3.0,NaN,2.0,2.0,2.0,2.0,3.0,2.0,NaN,NaN,61.0,NaN,200.0,3.0,1.0,64.0,142.0


In [60]:
df.rename({"_AGE80": "age", "CVDINFR4": "heart_attack", "CVDSTRK3": "stroke", "BPHIGH4": "high_bp", "BPMEDS": "bp_meds", "TOLDHI2": "high_chol", "_RACE": "race"}, axis = 1, inplace = True)
df.head()

,high_bp,bp_meds,sex,high_chol,heart_attack,stroke,diabetes,smoke_100,smoke_freq,drinking,age,daily_fruit,daily_veg,weekly_phys,race,height,weight,BMI
0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,63.0,0.50,2.17,3.0,1.0,70.0,280.0,40.259821
1,3.0,2.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,52.0,0.24,0.78,1.0,1.0,68.0,165.0,25.140621
3,1.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,63.0,1.00,0.20,3.0,1.0,67.0,180.0,28.250932
5,1.0,1.0,2.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,73.0,2.00,6.00,1.0,1.0,62.0,145.0,26.576299
6,1.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,70.0,1.29,1.14,1.0,4.0,66.0,148.0,23.937772


In [61]:
df.rename({"DIABETE3": 'diabetes', "SEX": "sex", "SMOKE100": "smoke_100", "SMOKDAY2": "smoke_freq", "DRNK3GE5": 'drinking', "_FRUTSUM": "daily_fruit", "_VEGESUM": "daily_veg", "_PA150R2": "weekly_phys"}, inplace = True, axis = 1)
df.head()

,high_bp,bp_meds,sex,high_chol,heart_attack,stroke,diabetes,smoke_100,smoke_freq,drinking,age,daily_fruit,daily_veg,weekly_phys,race,height,weight,BMI
0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,63.0,0.50,2.17,3.0,1.0,70.0,280.0,40.259821
1,3.0,2.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,52.0,0.24,0.78,1.0,1.0,68.0,165.0,25.140621
3,1.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,63.0,1.00,0.20,3.0,1.0,67.0,180.0,28.250932
5,1.0,1.0,2.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,73.0,2.00,6.00,1.0,1.0,62.0,145.0,26.576299
6,1.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,70.0,1.29,1.14,1.0,4.0,66.0,148.0,23.937772


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296927 entries, 0 to 441455
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   high_bp       296927 non-null  float64
 1   bp_meds       296927 non-null  float64
 2   sex           296927 non-null  float64
 3   high_chol     296927 non-null  float64
 4   heart_attack  296927 non-null  float64
 5   stroke        296927 non-null  float64
 6   diabetes      296927 non-null  float64
 7   smoke_100     296927 non-null  float64
 8   smoke_freq    296927 non-null  float64
 9   drinking      296927 non-null  float64
 10  age           296927 non-null  float64
 11  daily_fruit   296927 non-null  float64
 12  daily_veg     296927 non-null  float64
 13  weekly_phys   296927 non-null  float64
 14  race          296927 non-null  float64
 15  height        296927 non-null  float64
 16  weight        296927 non-null  float64
 17  BMI           296927 non-null  float64
dtypes: f

In [63]:
df.describe()

,high_bp,bp_meds,sex,high_chol,heart_attack,stroke,diabetes,smoke_100,smoke_freq,drinking,age,daily_fruit,daily_veg,weekly_phys,race,height,weight,BMI
count,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,2.969270e+05,2.969270e+05,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000
mean,2.134733,1.629138,1.574360,1.573501,0.062931,0.042812,0.168186,1.559673,2.773345,0.473514,57.593041,1.374188e+00,1.972965e+00,1.719487,1.552947,66.767347,179.994532,28.333837
std,1.003671,0.483036,0.494441,0.494569,0.242840,0.202433,0.374032,0.496427,0.602512,2.407998,15.381330,1.130886e+00,1.326744e+00,0.860221,1.354358,4.097215,47.575121,6.640913
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,18.000000,5.397605e-81,5.397605e-81,1.000000,1.000000,32.000000,15.454545,3.012282
25%,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,47.000000,5.700000e-01,1.140000e+00,1.000000,1.000000,64.000000,148.000000,24.081192
50%,3.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,60.000000,1.030000e+00,1.710000e+00,1.000000,1.000000,66.000000,175.000000,27.319164
75%,3.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,69.000000,2.000000e+00,2.490000e+00,3.000000,1.000000,70.000000,200.000000,31.258831
max,4.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,3.000000,30.000000,80.000000,1.900000e+01,1.950000e+01,3.000000,9.000000,91.000000,700.000000,182.329022


In [64]:
df = df[df["height"] <= 96]
df = df[df["weight"] <= 998]
df.describe()

,high_bp,bp_meds,sex,high_chol,heart_attack,stroke,diabetes,smoke_100,smoke_freq,drinking,age,daily_fruit,daily_veg,weekly_phys,race,height,weight,BMI
count,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,2.969270e+05,2.969270e+05,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000
mean,2.134733,1.629138,1.574360,1.573501,0.062931,0.042812,0.168186,1.559673,2.773345,0.473514,57.593041,1.374188e+00,1.972965e+00,1.719487,1.552947,66.767347,179.994532,28.333837
std,1.003671,0.483036,0.494441,0.494569,0.242840,0.202433,0.374032,0.496427,0.602512,2.407998,15.381330,1.130886e+00,1.326744e+00,0.860221,1.354358,4.097215,47.575121,6.640913
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,18.000000,5.397605e-81,5.397605e-81,1.000000,1.000000,32.000000,15.454545,3.012282
25%,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,47.000000,5.700000e-01,1.140000e+00,1.000000,1.000000,64.000000,148.000000,24.081192
50%,3.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,60.000000,1.030000e+00,1.710000e+00,1.000000,1.000000,66.000000,175.000000,27.319164
75%,3.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,69.000000,2.000000e+00,2.490000e+00,3.000000,1.000000,70.000000,200.000000,31.258831
max,4.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,3.000000,30.000000,80.000000,1.900000e+01,1.950000e+01,3.000000,9.000000,91.000000,700.000000,182.329022


In [65]:
df["smoke_freq"].fillna(3, inplace = True)
df["bp_meds"].fillna(2, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296927 entries, 0 to 441455
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   high_bp       296927 non-null  float64
 1   bp_meds       296927 non-null  float64
 2   sex           296927 non-null  float64
 3   high_chol     296927 non-null  float64
 4   heart_attack  296927 non-null  float64
 5   stroke        296927 non-null  float64
 6   diabetes      296927 non-null  float64
 7   smoke_100     296927 non-null  float64
 8   smoke_freq    296927 non-null  float64
 9   drinking      296927 non-null  float64
 10  age           296927 non-null  float64
 11  daily_fruit   296927 non-null  float64
 12  daily_veg     296927 non-null  float64
 13  weekly_phys   296927 non-null  float64
 14  race          296927 non-null  float64
 15  height        296927 non-null  float64
 16  weight        296927 non-null  float64
 17  BMI           296927 non-null  float64
dtypes: f

In [66]:
df["drinking"].fillna(88, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 296927 entries, 0 to 441455
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   high_bp       296927 non-null  float64
 1   bp_meds       296927 non-null  float64
 2   sex           296927 non-null  float64
 3   high_chol     296927 non-null  float64
 4   heart_attack  296927 non-null  float64
 5   stroke        296927 non-null  float64
 6   diabetes      296927 non-null  float64
 7   smoke_100     296927 non-null  float64
 8   smoke_freq    296927 non-null  float64
 9   drinking      296927 non-null  float64
 10  age           296927 non-null  float64
 11  daily_fruit   296927 non-null  float64
 12  daily_veg     296927 non-null  float64
 13  weekly_phys   296927 non-null  float64
 14  race          296927 non-null  float64
 15  height        296927 non-null  float64
 16  weight        296927 non-null  float64
 17  BMI           296927 non-null  float64
dtypes: f

In [67]:
df.dropna(inplace = True)

In [68]:
df["BMI"] = df["weight"] / 2.2 / (df["height"] * 0.0254) ** 2
df.head()

,high_bp,bp_meds,sex,high_chol,heart_attack,stroke,diabetes,smoke_100,smoke_freq,drinking,age,daily_fruit,daily_veg,weekly_phys,race,height,weight,BMI
0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,63.0,0.50,2.17,3.0,1.0,70.0,280.0,40.259821
1,3.0,2.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,52.0,0.24,0.78,1.0,1.0,68.0,165.0,25.140621
3,1.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,63.0,1.00,0.20,3.0,1.0,67.0,180.0,28.250932
5,1.0,1.0,2.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,73.0,2.00,6.00,1.0,1.0,62.0,145.0,26.576299
6,1.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,70.0,1.29,1.14,1.0,4.0,66.0,148.0,23.937772


In [69]:
df["drinking"].replace({88: 0}, inplace = True)
df = df[df["high_bp"] // 6.0 < 1]
df = df[df["high_chol"] // 6.0 < 1]
df = df[df["smoke_100"] // 6.0 < 1]
df = df[df["smoke_freq"] // 6.0 < 1]
df = df[df["heart_attack"] // 6.0 < 1]
df = df[df["stroke"] // 6.0 < 1]
df = df[df["diabetes"] // 6.0 < 1]
df = df[df["weekly_phys"] // 6.0 < 1]
df["daily_veg"] = df["daily_veg"] / 100
df["daily_fruit"] = df["daily_fruit"] / 100
df.describe()

,high_bp,bp_meds,sex,high_chol,heart_attack,stroke,diabetes,smoke_100,smoke_freq,drinking,age,daily_fruit,daily_veg,weekly_phys,race,height,weight,BMI
count,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,2.969270e+05,2.969270e+05,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000
mean,2.134733,1.629138,1.574360,1.573501,0.062931,0.042812,0.168186,1.559673,2.773345,0.473514,57.593041,1.374188e-02,1.972965e-02,1.719487,1.552947,66.767347,179.994532,28.333837
std,1.003671,0.483036,0.494441,0.494569,0.242840,0.202433,0.374032,0.496427,0.602512,2.407998,15.381330,1.130886e-02,1.326744e-02,0.860221,1.354358,4.097215,47.575121,6.640913
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,18.000000,5.397605e-83,5.397605e-83,1.000000,1.000000,32.000000,15.454545,3.012282
25%,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,47.000000,5.700000e-03,1.140000e-02,1.000000,1.000000,64.000000,148.000000,24.081192
50%,3.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,60.000000,1.030000e-02,1.710000e-02,1.000000,1.000000,66.000000,175.000000,27.319164
75%,3.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,69.000000,2.000000e-02,2.490000e-02,3.000000,1.000000,70.000000,200.000000,31.258831
max,4.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,3.000000,30.000000,80.000000,1.900000e-01,1.950000e-01,3.000000,9.000000,91.000000,700.000000,182.329022


In [70]:
df = df[df["daily_veg"]  < 20]
df = df[df["daily_fruit"] < 20]
df = df[df["bp_meds"] // 6.0 < 1]
df.describe()

,high_bp,bp_meds,sex,high_chol,heart_attack,stroke,diabetes,smoke_100,smoke_freq,drinking,age,daily_fruit,daily_veg,weekly_phys,race,height,weight,BMI
count,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,2.969270e+05,2.969270e+05,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000
mean,2.134733,1.629138,1.574360,1.573501,0.062931,0.042812,0.168186,1.559673,2.773345,0.473514,57.593041,1.374188e-02,1.972965e-02,1.719487,1.552947,66.767347,179.994532,28.333837
std,1.003671,0.483036,0.494441,0.494569,0.242840,0.202433,0.374032,0.496427,0.602512,2.407998,15.381330,1.130886e-02,1.326744e-02,0.860221,1.354358,4.097215,47.575121,6.640913
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,18.000000,5.397605e-83,5.397605e-83,1.000000,1.000000,32.000000,15.454545,3.012282
25%,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,47.000000,5.700000e-03,1.140000e-02,1.000000,1.000000,64.000000,148.000000,24.081192
50%,3.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,60.000000,1.030000e-02,1.710000e-02,1.000000,1.000000,66.000000,175.000000,27.319164
75%,3.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,69.000000,2.000000e-02,2.490000e-02,3.000000,1.000000,70.000000,200.000000,31.258831
max,4.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,3.000000,30.000000,80.000000,1.900000e-01,1.950000e-01,3.000000,9.000000,91.000000,700.000000,182.329022


In [71]:
df = df[df["drinking"] <= 30]

In [72]:
df.describe()

,high_bp,bp_meds,sex,high_chol,heart_attack,stroke,diabetes,smoke_100,smoke_freq,drinking,age,daily_fruit,daily_veg,weekly_phys,race,height,weight,BMI
count,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,2.969270e+05,2.969270e+05,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000
mean,2.134733,1.629138,1.574360,1.573501,0.062931,0.042812,0.168186,1.559673,2.773345,0.473514,57.593041,1.374188e-02,1.972965e-02,1.719487,1.552947,66.767347,179.994532,28.333837
std,1.003671,0.483036,0.494441,0.494569,0.242840,0.202433,0.374032,0.496427,0.602512,2.407998,15.381330,1.130886e-02,1.326744e-02,0.860221,1.354358,4.097215,47.575121,6.640913
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,18.000000,5.397605e-83,5.397605e-83,1.000000,1.000000,32.000000,15.454545,3.012282
25%,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,47.000000,5.700000e-03,1.140000e-02,1.000000,1.000000,64.000000,148.000000,24.081192
50%,3.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,60.000000,1.030000e-02,1.710000e-02,1.000000,1.000000,66.000000,175.000000,27.319164
75%,3.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,69.000000,2.000000e-02,2.490000e-02,3.000000,1.000000,70.000000,200.000000,31.258831
max,4.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,3.000000,30.000000,80.000000,1.900000e-01,1.950000e-01,3.000000,9.000000,91.000000,700.000000,182.329022


In [73]:
df = df[df["BMI"] < 200]

In [74]:
df.describe()

,high_bp,bp_meds,sex,high_chol,heart_attack,stroke,diabetes,smoke_100,smoke_freq,drinking,age,daily_fruit,daily_veg,weekly_phys,race,height,weight,BMI
count,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000,2.969270e+05,2.969270e+05,296927.000000,296927.000000,296927.000000,296927.000000,296927.000000
mean,2.134733,1.629138,1.574360,1.573501,0.062931,0.042812,0.168186,1.559673,2.773345,0.473514,57.593041,1.374188e-02,1.972965e-02,1.719487,1.552947,66.767347,179.994532,28.333837
std,1.003671,0.483036,0.494441,0.494569,0.242840,0.202433,0.374032,0.496427,0.602512,2.407998,15.381330,1.130886e-02,1.326744e-02,0.860221,1.354358,4.097215,47.575121,6.640913
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,18.000000,5.397605e-83,5.397605e-83,1.000000,1.000000,32.000000,15.454545,3.012282
25%,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,47.000000,5.700000e-03,1.140000e-02,1.000000,1.000000,64.000000,148.000000,24.081192
50%,3.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,60.000000,1.030000e-02,1.710000e-02,1.000000,1.000000,66.000000,175.000000,27.319164
75%,3.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,2.000000,3.000000,0.000000,69.000000,2.000000e-02,2.490000e-02,3.000000,1.000000,70.000000,200.000000,31.258831
max,4.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,3.000000,30.000000,80.000000,1.900000e-01,1.950000e-01,3.000000,9.000000,91.000000,700.000000,182.329022


In [75]:
df.head()

,high_bp,bp_meds,sex,high_chol,heart_attack,stroke,diabetes,smoke_100,smoke_freq,drinking,age,daily_fruit,daily_veg,weekly_phys,race,height,weight,BMI
0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,63.0,0.0050,0.0217,3.0,1.0,70.0,280.0,40.259821
1,3.0,2.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,52.0,0.0024,0.0078,1.0,1.0,68.0,165.0,25.140621
3,1.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,63.0,0.0100,0.0020,3.0,1.0,67.0,180.0,28.250932
5,1.0,1.0,2.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,73.0,0.0200,0.0600,1.0,1.0,62.0,145.0,26.576299
6,1.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,70.0,0.0129,0.0114,1.0,4.0,66.0,148.0,23.937772


In [76]:
df["stroke"].replace({2: 0}, inplace = True)
df["heart_attack"].replace({2: 0}, inplace = True)
df["diabetes"].replace({2: 1, 4: 1, 3: 0}, inplace = True)

In [ ]:
df_age50to70 = df[(df["age"] >= 50) & (df["age"] <= 70)]
df_age50to70

In [79]:
test_model(1, [32, 32, 32], [1, 1, 1, 1], "stroke", 5)

Epoch 1/20
928/928 [==============================] - 1s 656us/step - loss: 0.6302 - acc: 0.9384
Epoch 2/20
928/928 [==============================] - 1s 606us/step - loss: 0.3590 - acc: 0.9481
Epoch 3/20
928/928 [==============================] - 1s 599us/step - loss: 0.2682 - acc: 0.9534
Epoch 4/20
928/928 [==============================] - 1s 609us/step - loss: 0.2284 - acc: 0.9548
Epoch 5/20
928/928 [==============================] - 1s 591us/step - loss: 0.2020 - acc: 0.9563
Epoch 6/20
928/928 [==============================] - 1s 589us/step - loss: 0.1911 - acc: 0.9566
Epoch 7/20
928/928 [==============================] - 1s 596us/step - loss: 0.1833 - acc: 0.9568
Epoch 8/20
928/928 [==============================] - 1s 597us/step - loss: 0.1826 - acc: 0.9568
Epoch 9/20
928/928 [==============================] - 1s 594us/step - loss: 0.1783 - acc: 0.9568
Epoch 10/20
928/928 [==============================] - 1s 588us/step - loss: 0.1755 - acc: 0.9569
Epoch 11/20
928/928 [========

array([0.95842384, 0.95743033, 0.95692515, 0.95722826, 0.9566052 ])

In [ ]:
test_model(1, [32, 32, 32], [1, 1, 1, 1], "heart_attack", 5)

Epoch 1/20
928/928 [==============================] - 1s 605us/step - loss: 1.0488 - acc: 0.9020
Epoch 2/20
928/928 [==============================] - 1s 594us/step - loss: 0.3548 - acc: 0.9268
Epoch 3/20
928/928 [==============================] - 1s 587us/step - loss: 0.2641 - acc: 0.9345
Epoch 4/20
928/928 [==============================] - 1s 592us/step - loss: 0.2425 - acc: 0.9358
Epoch 5/20
928/928 [==============================] - 1s 623us/step - loss: 0.2326 - acc: 0.9362
Epoch 6/20
928/928 [==============================] - 1s 627us/step - loss: 0.2244 - acc: 0.9369
Epoch 7/20
928/928 [==============================] - 1s 622us/step - loss: 0.2206 - acc: 0.9371
Epoch 8/20
928/928 [==============================] - 1s 597us/step - loss: 0.2179 - acc: 0.9373
Epoch 9/20
928/928 [==============================] - 1s 604us/step - loss: 0.2173 - acc: 0.9372
Epoch 10/20
928/928 [==============================] - 1s 622us/step - loss: 0.2138 - acc: 0.9374
Epoch 11/20
928/928 [========

array([0.93735792, 0.93713901, 0.93703797, 0.93921024, 0.93762735])

In [56]:
test_model(1, [32, 32, 32], [1, 1, 1, 1], "diabetes", 5)

Epoch 1/20
928/928 [==============================] - 1s 628us/step - loss: 1.6952 - acc: 0.7731
Epoch 2/20
928/928 [==============================] - 1s 625us/step - loss: 0.4401 - acc: 0.8250
Epoch 3/20
928/928 [==============================] - 1s 592us/step - loss: 0.4160 - acc: 0.8292
Epoch 4/20
928/928 [==============================] - 1s 628us/step - loss: 0.4093 - acc: 0.8307
Epoch 5/20
928/928 [==============================] - 1s 646us/step - loss: 0.4053 - acc: 0.8307
Epoch 6/20
928/928 [==============================] - 1s 608us/step - loss: 0.4028 - acc: 0.8306
Epoch 7/20
928/928 [==============================] - 1s 588us/step - loss: 0.4011 - acc: 0.8308
Epoch 8/20
928/928 [==============================] - 1s 587us/step - loss: 0.4003 - acc: 0.8310
Epoch 9/20
928/928 [==============================] - 1s 585us/step - loss: 0.3997 - acc: 0.8311
Epoch 10/20
928/928 [==============================] - 1s 632us/step - loss: 0.3982 - acc: 0.8315
Epoch 11/20
928/928 [========

array([0.83276922, 0.83287025, 0.83015913, 0.83578345, 0.70371306])

In [47]:
df["diabetes"].value_counts()

3.0    246988
1.0     42095
4.0      5546
2.0      2298
Name: diabetes, dtype: int64

In [51]:
def test_model(hidden_layers, nodes, biases, disease, folds = 5):
    # assert hidden_layers == len(nodes) - 1 or hidden_layers == 0
    # assert hidden_layers == len(biases) - 2 or hidden_layers == 0
    ##Create func_model from params
    tf.random.set_seed(42)
    func_model = Sequential()
    ##Input layer
    func_model.add(Dense(nodes[0], use_bias = biases[0], input_dim = 15))
    ##Hidden layers
    if hidden_layers != 0:
        for i in range(hidden_layers - 2):
            func_model.add(Dense(nodes[i + 1], use_bias = biases[i + 1]))
    ##Output layers
    func_model.add(Dense(1, use_bias=biases[hidden_layers + 1], activation='sigmoid'))
    func_model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.001, momentum=0.0, nesterov=False, name="SGD"), metrics=['acc'])
    accuracy = np.ones(folds)
    #Splits for cross validation
    shuffled = df.sample(len(df.index), replace=True)
    fold_size = len(shuffled.index) // folds
    ##Cross-validation
    for i in range(folds):
        test_fold = shuffled.iloc[i * fold_size:(i + 1) * fold_size]
        rest = shuffled.iloc[(i + 1) * fold_size:]
        test_y = test_fold[disease]
        rest_y = rest[disease]
        vec = DictVectorizer()
        train_dict = rest.drop(_diseases, axis = 1).to_dict('records')
        train_vectorized = vec.fit_transform(train_dict).toarray()
        test_dict = test_fold.drop(_diseases, axis = 1).to_dict('records')
        test_vectorized = vec.fit_transform(test_dict).toarray()
        func_model.fit(train_vectorized, rest_y, epochs=20, batch_size=256, verbose = 1)
        test_pred = func_model.predict(test_vectorized)
        test_pred = test_pred.reshape(-1)
        accuracy[i] = np.count_nonzero(np.round(test_y) == np.round(test_pred)) / len(test_pred)
    return accuracy

In [59]:
# old people
test_model_age_filtered(1, [32, 32, 32], [1, 1, 1, 1], "stroke", 5)

Epoch 1/20
460/460 [==============================] - 1s 676us/step - loss: 0.9050 - acc: 0.9373
Epoch 2/20
460/460 [==============================] - 0s 608us/step - loss: 0.5978 - acc: 0.9428
Epoch 3/20
460/460 [==============================] - 0s 608us/step - loss: 0.4144 - acc: 0.9486
Epoch 4/20
460/460 [==============================] - 0s 607us/step - loss: 0.3038 - acc: 0.9542
Epoch 5/20
460/460 [==============================] - 0s 752us/step - loss: 0.2828 - acc: 0.9545
Epoch 6/20
460/460 [==============================] - 0s 812us/step - loss: 0.2292 - acc: 0.9571
Epoch 7/20
460/460 [==============================] - 0s 632us/step - loss: 0.2066 - acc: 0.9580
Epoch 8/20
460/460 [==============================] - 0s 574us/step - loss: 0.2114 - acc: 0.9578
Epoch 9/20
460/460 [==============================] - 0s 584us/step - loss: 0.1961 - acc: 0.9583
Epoch 10/20
460/460 [==============================] - 0s 572us/step - loss: 0.1987 - acc: 0.9581
Epoch 11/20
460/460 [========

array([0.95723035, 0.95944199, 0.95903368, 0.95811501, 0.9579789 ])

In [57]:
def test_model_age_filtered(hidden_layers, nodes, biases, disease, folds = 5):
    # assert hidden_layers == len(nodes) - 1 or hidden_layers == 0
    # assert hidden_layers == len(biases) - 2 or hidden_layers == 0
    ##Create func_model from params
    tf.random.set_seed(42)
    func_model = Sequential()
    ##Input layer
    func_model.add(Dense(nodes[0], use_bias = biases[0], input_dim = 15))
    ##Hidden layers
    if hidden_layers != 0:
        for i in range(hidden_layers - 2):
            func_model.add(Dense(nodes[i + 1], use_bias = biases[i + 1]))
    ##Output layers
    func_model.add(Dense(1, use_bias=biases[hidden_layers + 1], activation='sigmoid'))
    func_model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.001, momentum=0.0, nesterov=False, name="SGD"), metrics=['acc'])
    accuracy = np.ones(folds)
    #Splits for cross validation
    shuffled = df_age50to70.sample(len(df_age50to70.index), replace=True)
    fold_size = len(shuffled.index) // folds
    ##Cross-validation
    for i in range(folds):
        test_fold = shuffled.iloc[i * fold_size:(i + 1) * fold_size]
        rest = shuffled.iloc[(i + 1) * fold_size:]
        test_y = test_fold[disease]
        rest_y = rest[disease]
        vec = DictVectorizer()
        train_dict = rest.drop(_diseases, axis = 1).to_dict('records')
        train_vectorized = vec.fit_transform(train_dict).toarray()
        test_dict = test_fold.drop(_diseases, axis = 1).to_dict('records')
        test_vectorized = vec.fit_transform(test_dict).toarray()
        func_model.fit(train_vectorized, rest_y, epochs=20, batch_size=256, verbose = 1)
        test_pred = func_model.predict(test_vectorized)
        test_pred = test_pred.reshape(-1)
        accuracy[i] = np.count_nonzero(np.round(test_y) == np.round(test_pred)) / len(test_pred)
    return accuracy

In [90]:
def test_diabetes_model(hidden_layers, nodes, biases, disease = 'diabetes', folds = 5):
    # assert hidden_layers == len(nodes) - 1 or hidden_layers == 0
    # assert hidden_layers == len(biases) - 2 or hidden_layers == 0
    ##Create func_model from params
    tf.random.set_seed(42)
    func_model = Sequential()
    ##Input layer
    func_model.add(Dense(nodes[0], use_bias = biases[0], input_dim = 15))
    ##Hidden layers
    if hidden_layers != 0:
        for i in range(hidden_layers - 2):
            func_model.add(Dense(nodes[i + 1], use_bias = biases[i + 1]))
    ##Output layers
    func_model.add(Dense(3, use_bias=biases[hidden_layers + 1], activation='sigmoid'))
    func_model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.001, momentum=0.0, nesterov=False, name="SGD"), metrics=['acc'])
    accuracy = np.ones(folds)
    #Splits for cross validation
    shuffled = df.sample(len(df.index), replace=True)
    shuffled["diabetes"].replace({1: 'Yes', 2: 'Kinda', 3: 'No', 4: 'Kinda'}, inplace = True)
    fold_size = len(shuffled.index) // folds
    ##Cross-validation
    for i in range(folds):
        test_fold = shuffled.iloc[i * fold_size:(i + 1) * fold_size]
        rest = shuffled.iloc[(i + 1) * fold_size:]
        test_y = test_fold[[disease]]
        rest_y = rest[[disease]]
        vec = DictVectorizer()
        test_y_dict = test_y.to_dict('records')
        test_y_vec = vec.fit_transform(test_y_dict).toarray()
        rest_y_dict = test_y.to_dict('records')
        rest_y_vec = vec.fit_transform(rest_y_dict).toarray()
        train_dict = rest.drop(_diseases, axis = 1).to_dict('records')
        train_vectorized = vec.fit_transform(train_dict).toarray()
        test_dict = test_fold.drop(_diseases, axis = 1).to_dict('records')
        test_vectorized = vec.fit_transform(test_dict).toarray()
        func_model.fit(train_vectorized, rest_y, epochs=20, batch_size=256, verbose = 1)
        test_pred = func_model.predict(test_vectorized)
        test_pred = test_pred.reshape(-1)
        accuracy[i] = np.count_nonzero(np.round(test_y) == np.round(test_pred)) / len(test_pred)
    return accuracy

In [91]:
test_diabetes_model(2, [32, 32, 32], [1, 1, 1, 1])

Epoch 1/20


ValueError: in user code:

    File "/Users/willrathgeb/opt/anaconda3/envs/deep_learning/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/willrathgeb/opt/anaconda3/envs/deep_learning/lib/python3.10/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/willrathgeb/opt/anaconda3/envs/deep_learning/lib/python3.10/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/willrathgeb/opt/anaconda3/envs/deep_learning/lib/python3.10/site-packages/keras/engine/training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/willrathgeb/opt/anaconda3/envs/deep_learning/lib/python3.10/site-packages/keras/engine/training.py", line 948, in compute_loss
        return self.co

,high_bp,bp_meds,sex,high_chol,heart_attack,stroke,diabetes,smoke_100,smoke_freq,drinking,age,daily_fruit,daily_veg,weekly_phys,race,height,weight,BMI
0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,63.0,0.50,2.17,3.0,1.0,70.0,280.0,40.259821
1,3.0,2.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,52.0,0.24,0.78,1.0,1.0,68.0,165.0,25.140621
3,1.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,63.0,1.00,0.20,3.0,1.0,67.0,180.0,28.250932
6,1.0,1.0,2.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,70.0,1.29,1.14,1.0,4.0,66.0,148.0,23.937772
9,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,68.0,1.14,3.14,1.0,1.0,67.0,161.0,25.268890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441441,1.0,1.0,1.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,63.0,0.14,1.71,3.0,5.0,72.0,230.0,31.258831
441442,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,3.0,0.0,59.0,0.29,0.85,3.0,5.0,66.0,145.0,23.452547
441445,3.0,2.0,1.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,68.0,0.60,1.07,3.0,5.0,75.0,220.0,27.555611
441454,1.0,1.0,1.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,52.0,2.00,1.35,3.0,5.0,71.0,166.0,23.200710
